# Scraper completo

In [1]:
def articles(s_nota):
    art_dict = {}
    #Indicar la sección a la cual pertenece la nota
    #art_dict['seccion'] = section.a.get_text()
    #Extraer Título de la nota
    titulo = s_nota.find('h1')
    if titulo:
        art_dict['titulo'] = titulo.text
    else:
        art_dict['titulo'] = None
    #Extraer fecha
    fecha_str = s_nota.find('time').get('datetime') #Retorna un string de la fecha en formato ISO
    fecha = datetime.fromisoformat(fecha_str).date() #Quita el formato ISO de la fecha y retorna solo la fecha
    if fecha:
        art_dict['fecha'] = fecha.strftime("%d %b, %Y")
    else:
        art_dict['fecha'] = None
    #Extraer bajada
    bajada = s_nota.find('h3')
    if bajada:
        art_dict['bajada'] = bajada.text
    else:
        art_dict['bajada'] = None
    #Extraer Volanta
    volanta = s_nota.find('h4')
    if volanta:
        art_dict['volanta'] = volanta.text
    else:
        art_dict['volanta'] = None
    #Extraer cuerpo
    cuerpo = s_nota.find('div', attrs={'class':'article-main-content article-text'})
    if cuerpo:
        art_dict['cuerpo'] = cuerpo.text
    else:
        art_dict['cuerpo'] = None
    #Extraer Autor
    autor = s_nota.find('div', attrs={'class':'author-name'})
    if autor:
        art_dict['autor'] = autor.text.replace('Por ','')
    else:
        art_dict['autor'] = None
    #Extraer imagen
    media = s_nota.find('figure', attrs={'class':'object-fit-block--contain intrinsic-container intrinsic-container-3x2'})
    if media:
        imagenes = media.find_all('img')
        if len(imagenes) == 0:
            print('no hay imagenes')
        else:
            imagen = imagenes[-1].get('src')
            try:
                img_request = requests.get(imagen)
                if img_request.status_code == 200:
                    art_dict['imagen'] = img_request.content
                else:
                    art_dict['imagen'] = None
            except:
                print('No se encontró imagen')
    else:
        print('No se encontro media')
    
    return art_dict

In [2]:
def scraper(link):
    try:
        nota = requests.get(link)
    except Exception as e:
        print('Error scrapeando URL ', link)
        return None
    
    if nota.status_code !=200:
        print(f'Error obteniendo nota {link}')
        print(f'Status code {link.status_code}')
        return None
    s_nota = BeautifulSoup(nota.text, 'lxml')
    art_dict = articles(s_nota)
    art_dict['url'] = link
    
    return art_dict

In [3]:
def get_links_noticias(s_sec):
    try:
        main_article = s_sec.find('section', attrs={'class':'top-content'}).find_all('h2')
        main_article_list = [url[0:-1]+ma.find('a').get('href') for ma in main_article]
    except:
        main_article_list=[]
    try:
        featured_articles = s_sec.find_all('article', attrs={'class':'article-item--featured'})
        featured_article_list = [url[0:-1]+fa.find('h3').find('a').get('href') for fa in featured_articles]
    except:
        featured_article_list = []
            
    try:
        regular_articles = s_sec.find_all('article', attrs={'class':'article-item article-item--teaser'})
        regular_articles_list = [url[0:-1]+ra.find('h4').find('a').get('href') for ra in regular_articles]
    except:
        regular_article_list = []
            
    try:
        cultura = s_sec.find_all('div', attrs={'class':'article-box__container'})
        cultura_list = [url[0:-1]+c.find('h2').find('a').get('href') for c in cultura]
    except:
        cultura_list = []
    
    links = main_article_list + featured_article_list + regular_articles_list + cultura_list

    return links

In [4]:
#Importar librerias a utilizar
import requests
from bs4 import BeautifulSoup
from datetime import datetime
from IPython.display import Image

#Definimos la url principal
url = 'https://www.pagina12.com.ar/'

#Definir un objeto de la clase Response, el cual nos retorna el código de respuesta al hacer request a la url
p12 = requests.get(url)

try:
    if p12.status_code == 200:
        #Transformamos el texto con etiquetas html a un objeto BeautifulSoup, parseandolo con lxml 
        s = BeautifulSoup(p12.text, 'lxml')
        #Extraemos las secciones del diario
        #Usamos el método find en el objeto 's' y definimos la etiqueta está la infomación que se quiere extraer.
        #Estos se encuentran en una lista 'ul'. Al indicar los atributos como un diccionario, indicamos la clase
        #especifica de la ul que queremos.Finalmente, con el método find_all('li'), lo que hacemos es que nos retorne
        #todas las etiquetas li dentro de esta ul, como una lista
        secciones = s.find('ul', attrs={'class':'horizontal-list main-sections hide-on-dropdown'}).find_all('li')
        #Obtenemos los links de cada una de las secciones
        #La forma en que se hará es equivalente a link_secciones = [seccion.a.get('href') for seccion in secciones]
        
    else:
        raise ValueError(f'Error: {p12.status_code}')

except ValueError as ve:
    print(ve)
    print("\n")

link_secciones = []
for seccion in secciones:
    link_secciones.append(seccion.a.get('href'))
notas = []
for link in link_secciones:
    try:
        r = requests.get(link)
        if r.status_code == 200:
            s_link = BeautifulSoup(r.text, 'lxml')
            notas.extend(get_links_noticias(s_link)) 
        else:
            print('No se encontro la seccion ', link)
    except:
        print('No se encontro la seccion ', link)

data =[]
for i, nota in enumerate(notas):
    print(f'Scrapeando nota {i+1}/{len(notas)}')
    data.append(scraper(nota))






Scrapeando nota 1/64
Scrapeando nota 2/64
Scrapeando nota 3/64
Scrapeando nota 4/64
Scrapeando nota 5/64
Scrapeando nota 6/64
Scrapeando nota 7/64
Scrapeando nota 8/64
Scrapeando nota 9/64
Scrapeando nota 10/64
Scrapeando nota 11/64
Scrapeando nota 12/64
Scrapeando nota 13/64
Scrapeando nota 14/64
Scrapeando nota 15/64
Scrapeando nota 16/64
Scrapeando nota 17/64
Scrapeando nota 18/64
Scrapeando nota 19/64
Scrapeando nota 20/64
Scrapeando nota 21/64
Scrapeando nota 22/64
Scrapeando nota 23/64
Scrapeando nota 24/64
Scrapeando nota 25/64
Scrapeando nota 26/64
Scrapeando nota 27/64
Scrapeando nota 28/64
Scrapeando nota 29/64
Scrapeando nota 30/64
Scrapeando nota 31/64
Scrapeando nota 32/64
Scrapeando nota 33/64
Scrapeando nota 34/64
Scrapeando nota 35/64
Scrapeando nota 36/64
Scrapeando nota 37/64
Scrapeando nota 38/64
Scrapeando nota 39/64
Scrapeando nota 40/64
Scrapeando nota 41/64
Scrapeando nota 42/64
Scrapeando nota 43/64
Scrapeando nota 44/64
Scrapeando nota 45/64
Scrapeando nota 46/

In [5]:
len(data)

64

In [6]:
import pandas as pd

In [7]:
df = pd.DataFrame(data)
df

,titulo,fecha,bajada,volanta,cuerpo,autor,imagen,url
0,Alberto Fernández y otros presidentes latinoam...,"24 Aug, 2022","El Presidente junto a Luis Arce, Andrés Manuel...",Denunciaron que el juicio busca eliminar contr...,El presidente Alberto Fernández compartió un c...,None,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,https://www.pagina12.com.ar/476047-alberto-fer...
1,Diputados trabaja en la creación de Universida...,"24 Aug, 2022",,Reunión de comisión en vivo,"En una reunión conjunta, las comisiones de Edu...",None,"b'\xff\xd8\xff\xe1\x00""Exif\x00\x00MM\x00*\x00...",https://www.pagina12.com.ar/476045-diputados-t...
2,El emotivo video de La Cámpora para apoyar a C...,"24 Aug, 2022",La organización política compartió en sus rede...,"""20 veces te volveríamos a elegir""",La Cámpora reafirmó su apoyo a Cristina Fernán...,None,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,https://www.pagina12.com.ar/476032-el-emotivo-...
3,Cristina Kirchner y otro tuit que revela los v...,"24 Aug, 2022",,Un obiturario que deja plasmado la relación en...,Cristina Kirchner volvió a dejar plasmado el s...,None,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,https://www.pagina12.com.ar/476028-cristina-ki...
4,Organismos de Derechos Humanos manifestaron su...,"24 Aug, 2022","Abuelas de Plaza de Mayo, Madres de Plaza de M...",Llamaron a ganar la calle en defensa de la dem...,Organismos de Derechos Humanos se sumaron este...,None,b'\xff\xd8\xff\xe1\x00FExif\x00\x00II*\x00\x08...,https://www.pagina12.com.ar/476021-organismos-...
...,...,...,...,...,...,...,...,...
59,España se ofrece como sede de conversaciones d...,"24 Aug, 2022",,Petro le dijo a Sánchez que necesita de la otr...,"El presidente del Gobierno de España, Pedro Sá...",None,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,https://www.pagina12.com.ar/476110-espana-se-o...
60,"A días del plebiscito, el gobierno chileno señ...","24 Aug, 2022",Aunque el “Rechazo” lidera las encuestas (que ...,Se intensifican las campañas del Apruebo y el ...,"Desde SantiagoA diez días del plebiscito, dond...",Juan Carlos Ramírez Figueroa,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,https://www.pagina12.com.ar/476098-a-dias-del-...
61,Finlandia: Sanna Marín rompió en llanto por el...,"24 Aug, 2022","""A veces también aspiro a la alegría, a la luz...",El acoso de la prensa local a la primera ministra,"Desde hace algunas semanas, la primera ministr...",None,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01...,https://www.pagina12.com.ar/476087-finlandia-s...
62,Alemania reimpone el barbijo por posible rebro...,"24 Aug, 2022",El Gobierno anunció que será obligatorio en de...,Ante la llegada del otoño,El Gobierno de Alemania volvió a establecer es...,None,"b'\xff\xd8\xff\xe1\x00""Exif\x00\x00MM\x00*\x00...",https://www.pagina12.com.ar/476007-alemania-re...


In [8]:
df.to_csv('Platzi.csv')